In [ ]:
import numpy as np
import networkx as nx
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
from pgmpy.sampling import BayesianModelSampling
from scipy.stats import beta, dirichlet
from operator import add
import matplotlib.pyplot as plt

## True Causal Model From Environment:

In [ ]:
model = BayesianModel([('Enfermedad', 'Final'), ('Tratamiento', 'Final'), ('Tratamiento', 'Reaccion'),
                       ('Reaccion', 'Final')])

cpd_e = TabularCPD(variable='Enfermedad', variable_card=2, values=[[0.7, 0.3]])
cpd_tr = TabularCPD(variable='Tratamiento', variable_card=2, values=[[0.5, 0.5]])

cpd_r_tr = TabularCPD(variable='Reaccion', variable_card=2, 
                   values=[[.7,.4], [.3,.6]], #vivir|farmaco vivir|cirugia #morir|farmaco morir|cirugia
                   evidence=['Tratamiento'],
                   evidence_card=[2])

cpd_f_e_tr_r = TabularCPD(variable='Final', variable_card=2, 
                   values=[[.6,0,.8,0,.4,0,.9,0],
                           [.4,1,.2,1,.6,1,.1,1]], #vivir|vivir,farmaco,a #morir|
                  evidence=['Enfermedad', 'Tratamiento','Reaccion'],
                  evidence_card=[2, 2,2])

model.add_cpds(cpd_e, cpd_tr, cpd_r_tr, cpd_f_e_tr_r)

infer = VariableElimination(model)
sampling = BayesianModelSampling(model)

In [ ]:
class true_causal_model:
    def __init__(self,N,infer,sampling,model):
        self.num_variables=N
        self.infer=infer
        self.sampling=sampling
        self.true_model=model

    def action_simulator(self,chosen_action):
        response=dict()
        elements = [0,1]
        probabilities = [0.7,0.3]
        res=np.random.choice(elements, 1, p=probabilities)
        response['Enfermedad']= res[0]
        response['Tratamiento']=chosen_action
        response['Reaccion']=self.infer.map_query(['Reaccion'],evidence={'Tratamiento': response['Tratamiento'],'Enfermedad': response['Enfermedad']})['Reaccion']
        response['Final']=self.infer.map_query(['Final'],evidence={'Tratamiento': response['Tratamiento'],'Enfermedad': response['Enfermedad'],'Reaccion':response['Reaccion']})['Final']
        return(response)

In [ ]:
class fully_informed_agent:
    def __init__(self,model_simulator):
        self.beliefs=dict()
        self.beliefs['Enfermedad']=[0.7, #0
                                    0.3] #1
        
        self.beliefs['Tratamiento']=[0.5, #0
                                     0.5] #1
        
        self.beliefs['Reaccion|Tratamiento']=[[.7,.4], #0 
                                              [.3,.6]] #1
        
        self.beliefs['Final|Enfermedad, Tratamiento, Reaccion']=[[.6,0,.8,0,.4,0,.9,0], #0
                                                                 [.4,1,.2,1,.6,1,.1,1]] #1
        self.recompensa=[0]
        self.simulator=model_simulator.action_simulator
        
    def do_calculus(self,final,treatment): 
        #La distribución P(Final|do(Tratamiento))=P(Final|Enfermedad, Tratamiento, Reaccion)P(Reaccion|Tratamiento)P(Enfermedad)
        #Treatment = 1 ó 0
        #Quiero que me de la probabilidad de vivir=0 ó =1 dado treatment=0 ó =1
        if (treatment==0):
            res=self.beliefs['Final|Enfermedad, Tratamiento, Reaccion'][final][0]*self.beliefs['Reaccion|Tratamiento'][0][0]*self.beliefs['Enfermedad'][0] \
            +self.beliefs['Final|Enfermedad, Tratamiento, Reaccion'][final][1]*self.beliefs['Reaccion|Tratamiento'][1][0]*self.beliefs['Enfermedad'][0] \
            +self.beliefs['Final|Enfermedad, Tratamiento, Reaccion'][final][4]*self.beliefs['Reaccion|Tratamiento'][0][0]*self.beliefs['Enfermedad'][1] \
            +self.beliefs['Final|Enfermedad, Tratamiento, Reaccion'][final][5]*self.beliefs['Reaccion|Tratamiento'][1][0]*self.beliefs['Enfermedad'][1]
        else:
            res=self.beliefs['Final|Enfermedad, Tratamiento, Reaccion'][final][2]*self.beliefs['Reaccion|Tratamiento'][0][1]*self.beliefs['Enfermedad'][0] \
                +self.beliefs['Final|Enfermedad, Tratamiento, Reaccion'][final][3]*self.beliefs['Reaccion|Tratamiento'][1][1]*self.beliefs['Enfermedad'][0] \
                +self.beliefs['Final|Enfermedad, Tratamiento, Reaccion'][final][6]*self.beliefs['Reaccion|Tratamiento'][0][1]*self.beliefs['Enfermedad'][1] \
                +self.beliefs['Final|Enfermedad, Tratamiento, Reaccion'][final][7]*self.beliefs['Reaccion|Tratamiento'][1][1]*self.beliefs['Enfermedad'][1]
        return(res)
    
    def make_decision(self,final=1):
        f_0= self.do_calculus(final,0)
        f_1=self.do_calculus(final,1)
        if (f_0 > f_1):
            return(0)
        else:
            return(1)
        
    def observing_external(self):
        chosen_action=self.make_decision()
        datos=self.simulator(chosen_action)
        self.recompensa.append(datos['Final'])
        return(datos)

    def get_reward(self):
        return(self.recompensa)

In [ ]:
class causal_agent:
    def __init__(self, true_model,model_simulator, max_variables, max_valores):
        self.max_variables=max_variables
        self.local_model=true_model ## Este es el directamente creado con PGMPY, no la clase true_causal_model
        self.infer=VariableElimination(self.local_model)
        self.beliefs=dict()
        self.K=max_valores
        self.simulator=model_simulator.action_simulator
        
        self.alpha_enfermedad=np.random.rand(self.K).tolist()
        self.alpha_tratamiento=np.random.rand(self.K).tolist()
        self.alpha_reaccion_farmaco=np.random.rand(self.K).tolist()
        self.alpha_reaccion_cirugia=np.random.rand(self.K).tolist()
        self.alpha_A_f_v=np.random.rand(self.K).tolist()
        self.alpha_A_f_m=np.random.rand(self.K).tolist()
        self.alpha_A_c_v=np.random.rand(self.K).tolist()
        self.alpha_A_c_m=np.random.rand(self.K).tolist()
        self.alpha_B_f_v=np.random.rand(self.K).tolist()
        self.alpha_B_f_m=np.random.rand(self.K).tolist()
        self.alpha_B_c_v=np.random.rand(self.K).tolist()
        self.alpha_B_c_m=np.random.rand(self.K).tolist()
                
        self.conteo_enfermedad_A=0
        self.conteo_enfermedad_B=0
        self.conteo_tratamiento_f=0
        self.conteo_tratamiento_c=0
        self.conteo_reaccion_v_f=0
        self.conteo_reaccion_m_f=0
        self.conteo_reaccion_v_c=0
        self.conteo_reaccion_m_c=0
        
        self.conteo_v_A_f_v=0
        self.conteo_v_A_f_m=0
        self.conteo_v_A_c_v=0
        self.conteo_v_A_c_m=0
        self.conteo_v_B_f_v=0
        self.conteo_v_B_f_m=0
        self.conteo_v_B_c_v=0
        self.conteo_v_B_c_m=0
        self.conteo_m_A_f_v=0
        self.conteo_m_A_f_m=0
        self.conteo_m_A_c_v=0
        self.conteo_m_A_c_m=0
        self.conteo_m_B_f_v=0
        self.conteo_m_B_f_m=0
        self.conteo_m_B_c_v=0
        self.conteo_m_B_c_m=0
        
        self.recompensa=[0]
        self.rounds=0
    
    def belief_formation(self):
        alpha_enfermedad=self.alpha_enfermedad
        alpha_tratamiento=self.alpha_tratamiento
        
        alpha_reaccion_farmaco=self.alpha_reaccion_farmaco
        alpha_reaccion_cirugia=self.alpha_reaccion_cirugia
        
        alpha_A_f_v=self.alpha_A_f_v
        alpha_A_f_m=self.alpha_A_f_m
        alpha_A_c_v=self.alpha_A_c_v
        alpha_A_c_m=self.alpha_A_c_m
        alpha_B_f_v=self.alpha_B_f_v
        alpha_B_f_m=self.alpha_B_f_m
        alpha_B_c_v=self.alpha_B_c_v
        alpha_B_c_m=self.alpha_B_c_m
        
        
        lista=dirichlet.rvs(alpha_enfermedad,size=1).tolist()
        self.beliefs['Enfermedad']=lista[0]
        
        lista=dirichlet.rvs(alpha_tratamiento,size=1).tolist()
        self.beliefs['Tratamiento']=lista[0]
        
        lista_1=dirichlet.rvs(alpha_reaccion_farmaco,size=1).tolist()
        lista_2=dirichlet.rvs(alpha_reaccion_cirugia,size=1).tolist()
        lista_3=np.array([lista_1[0],lista_2[0]]).transpose()
        lista_4=lista_3.tolist()
        self.beliefs['Reaccion|Tratamiento']=lista_4
        
        lista_1=dirichlet.rvs(alpha_A_f_v,size=1).tolist()
        lista_2=dirichlet.rvs(alpha_A_f_m,size=1).tolist()
        lista_3=dirichlet.rvs(alpha_A_c_v,size=1).tolist()
        lista_4=dirichlet.rvs(alpha_A_c_m,size=1).tolist()
        lista_5=dirichlet.rvs(alpha_B_f_v,size=1).tolist()
        lista_6=dirichlet.rvs(alpha_B_f_m,size=1).tolist()
        lista_7=dirichlet.rvs(alpha_B_c_v,size=1).tolist()
        lista_8=dirichlet.rvs(alpha_B_c_m,size=1).tolist()
        lista_9=np.array([lista_1[0],lista_2[0],lista_3[0],lista_4[0],lista_5[0],lista_6[0],lista_7[0],lista_8[0]]).transpose()
        lista_10=lista_9.tolist()
        self.beliefs['Final|Enfermedad, Tratamiento, Reaccion']=lista_10
        print("Creencias formadas")
        return(self.beliefs)
    
    def print_beliefs(self):
        return(self.beliefs)
    
    def fix_causal_model(self):
        model1=self.local_model
        model=model1.copy()
        
        cpd_e=TabularCPD(variable='Enfermedad', variable_card=2, values=[self.beliefs['Enfermedad']])
        cpd_tr=TabularCPD(variable='Tratamiento', variable_card=2, values=[self.beliefs['Tratamiento']])
        cpd_r_tr=cpd_r_tr = TabularCPD(variable='Reaccion', variable_card=2, values=self.beliefs['Reaccion|Tratamiento'], evidence=['Tratamiento'],evidence_card=[2])
        cpd_f_e_tr_r= TabularCPD(variable='Final', variable_card=2, values=self.beliefs['Final|Enfermedad, Tratamiento, Reaccion'],evidence=['Enfermedad', 'Tratamiento','Reaccion'],evidence_card=[2, 2,2])
        model.add_cpds(cpd_e, cpd_tr, cpd_r_tr, cpd_f_e_tr_r)
        
        if (model.check_model()):
            self.local_model=model
            self.infer= VariableElimination(model)
            print("Modelo Causal local creado")
            return(model,infer)
        else:
            print("Error en CPT")
        
    def do_calculus(self,final,treatment): 
        #La distribución P(Final|do(Tratamiento))=P(Final|Enfermedad, Tratamiento, Reaccion)P(Reaccion|Tratamiento)P(Enfermedad)
        #Treatment = 1 ó 0
        #Quiero que me de la probabilidad de vivir=0 ó =1 dado treatment=0 ó =1
        if (treatment==0):
            res=self.beliefs['Final|Enfermedad, Tratamiento, Reaccion'][final][0]*self.beliefs['Reaccion|Tratamiento'][0][0]*self.beliefs['Enfermedad'][0] \
            +self.beliefs['Final|Enfermedad, Tratamiento, Reaccion'][final][1]*self.beliefs['Reaccion|Tratamiento'][1][0]*self.beliefs['Enfermedad'][0] \
            +self.beliefs['Final|Enfermedad, Tratamiento, Reaccion'][final][4]*self.beliefs['Reaccion|Tratamiento'][0][0]*self.beliefs['Enfermedad'][1] \
            +self.beliefs['Final|Enfermedad, Tratamiento, Reaccion'][final][5]*self.beliefs['Reaccion|Tratamiento'][1][0]*self.beliefs['Enfermedad'][1]
        else:
            res=self.beliefs['Final|Enfermedad, Tratamiento, Reaccion'][final][2]*self.beliefs['Reaccion|Tratamiento'][0][1]*self.beliefs['Enfermedad'][0] \
                +self.beliefs['Final|Enfermedad, Tratamiento, Reaccion'][final][3]*self.beliefs['Reaccion|Tratamiento'][1][1]*self.beliefs['Enfermedad'][0] \
                +self.beliefs['Final|Enfermedad, Tratamiento, Reaccion'][final][6]*self.beliefs['Reaccion|Tratamiento'][0][1]*self.beliefs['Enfermedad'][1] \
                +self.beliefs['Final|Enfermedad, Tratamiento, Reaccion'][final][7]*self.beliefs['Reaccion|Tratamiento'][1][1]*self.beliefs['Enfermedad'][1]
        return(res)
    
    def make_decision(self,final=1):
        f_0= self.do_calculus(final,0)
        f_1=self.do_calculus(final,1)
        if (f_0 > f_1):
            return(0)
        else:
            return(1)
        
    def observing_external(self):
        self.rounds=self.rounds+1
        chosen_action=self.make_decision()
        datos=self.simulator(chosen_action)
        self.recompensa.append(datos['Final'])
        return(datos)
        
    #def prob_data_given_beliefs(self,datos):
        #Dados los beliefs, que son distribuciones, genero un modelo gráfico causal
        #Este modelo gráfico causal da lugar a una distribución intervencional y puedo calcular
        #la probabilidad de ver esa observación dadas las dirichlet
        #result = [a * 8 for a in self.alpha]
        
        
    def belief_updating(self):
        datos=self.observing_external()
        lista_datos=list(datos.values())
        #self.alpha=list(map(add,self.alpha,self.conteo))
        
        #Conteo de enfermedad
        if(lista_datos[0]==0):
            self.conteo_enfermedad_A=self.conteo_enfermedad_A+1
            self.alpha_enfermedad[0]=self.alpha_enfermedad[0]+self.conteo_enfermedad_A
        else:
            self.conteo_enfermedad_B=self.conteo_enfermedad_B+1
            self.alpha_enfermedad[1]=self.alpha_enfermedad[0]+self.conteo_enfermedad_B
        
        #Conteo de tratamiento
        if(lista_datos[1]==0):
            self.conteo_tratamiento_f=self.conteo_tratamiento_f+1
            self.alpha_tratamiento[0]=self.alpha_tratamiento[0]+self.conteo_tratamiento_f
        else:
            self.conteo_tratamiento_c=self.conteo_tratamiento_c+1
            self.alpha_tratamiento[1]=self.alpha_tratamiento[0]+self.conteo_tratamiento_c
        
        #Conteo de reaccion dado tratamiento
        if(lista_datos[1]==0 & lista_datos[2]==0):
            self.conteo_reaccion_v_f=self.conteo_reaccion_v_f+1
            self.alpha_reaccion_farmaco[0]=self.alpha_reaccion_farmaco[0]+self.conteo_reaccion_v_f
        if(lista_datos[1]==0 & lista_datos[2]==1):
            self.conteo_reaccion_m_f=self.conteo_reaccion_m_f+1
            self.alpha_reaccion_farmaco[1]=self.alpha_reaccion_farmaco[1]+self.conteo_reaccion_m_f
        
        if(lista_datos[1]==1 & lista_datos[2]==0):
            self.conteo_reaccion_v_c=self.conteo_reaccion_v_c+1
            self.alpha_reaccion_cirugia[0]+self.conteo_reaccion_v_c
        if(lista_datos[1]==0 & lista_datos[2]==1):
            self.conteo_reaccion_m_c=self.conteo_reaccion_m_c+1
            self.alpha_reaccion_cirugia[1]=self.alpha_reaccion_cirugia[1]+self.conteo_reaccion_m_c
        
        #Conteo de vivir dado enfermedad, tratamiento y reaccion
        #Para vivir
        if(lista_datos[0]==0 & lista_datos[1]==0 & lista_datos[2]==0 & lista_datos[3]==0):
            self.conteo_v_A_f_v=self.conteo_v_A_f_v+1
            self.alpha_A_f_v[0]=self.alpha_A_f_v[0]+self.conteo_v_A_f_v
            
        if(lista_datos[0]==0 & lista_datos[1]==0 & lista_datos[2]==1 & lista_datos[3]==0):
            self.conteo_v_A_f_m=self.conteo_v_A_f_m+1
            self.alpha_A_f_m[0]=self.alpha_A_f_m[0]+self.conteo_v_A_f_m
            
        if(lista_datos[0]==0 & lista_datos[1]==1 & lista_datos[2]==0 & lista_datos[3]==0):
            self.conteo_v_A_c_v=self.conteo_v_A_c_v+1
            self.alpha_A_c_v[0]=self.alpha_A_c_v[0]+self.conteo_v_A_c_v
            
        if(lista_datos[0]==0 & lista_datos[1]==1 & lista_datos[2]==1 & lista_datos[3]==0):
            self.conteo_v_A_c_m=self.conteo_v_A_c_m+1
            self.alpha_A_c_m[0]=self.alpha_A_c_m[0]+self.conteo_v_A_c_m
        
        if(lista_datos[0]==1 & lista_datos[1]==0 & lista_datos[2]==0 & lista_datos[3]==0):
            self.conteo_v_B_f_v=self.conteo_v_B_f_v+1
            self.alpha_B_f_v[0]=self.alpha_B_f_v[0]+self.conteo_v_B_f_v
        
        if(lista_datos[0]==1 & lista_datos[1]==0 & lista_datos[2]==1 & lista_datos[3]==0):
            self.conteo_v_B_f_m=self.conteo_v_B_f_m+1
            self.alpha_B_f_m[0]=self.alpha_B_f_m[0]+self.conteo_v_B_f_m
            
        if(lista_datos[0]==1 & lista_datos[1]==1 & lista_datos[2]==0 & lista_datos[3]==0):
            self.conteo_v_B_c_v=self.conteo_v_B_c_v+1
            self.alpha_B_c_v[0]=self.alpha_B_c_v[0]+self.conteo_v_B_c_v
            
        if(lista_datos[0]==1 & lista_datos[1]==1 & lista_datos[2]==1 & lista_datos[3]==0):
            self.conteo_v_B_c_m=self.conteo_v_B_c_m+1
            self.alpha_B_c_m[0]=self.alpha_B_c_m[0]+self.conteo_v_B_c_m
        #Para morir
        if(lista_datos[0]==0 & lista_datos[1]==0 & lista_datos[2]==0 & lista_datos[3]==1):
            self.conteo_m_A_f_v=self.conteo_m_A_f_v+1
            self.alpha_A_f_v[1]=self.alpha_A_f_v[1]+self.conteo_m_A_f_v
            
        if(lista_datos[0]==0 & lista_datos[1]==0 & lista_datos[2]==1 & lista_datos[3]==1):     
            self.conteo_m_A_f_m=self.conteo_m_A_f_m+1
            self.alpha_A_f_m[1]=self.alpha_A_f_m[1]+self.conteo_m_A_f_m
            
        if(lista_datos[0]==0 & lista_datos[1]==1 & lista_datos[2]==0 & lista_datos[3]==1):    
            self.conteo_m_A_c_v=self.conteo_m_A_c_v+1
            self.alpha_A_c_v[1]=self.alpha_A_c_v[1]+self.conteo_m_A_c_v
            
        if(lista_datos[0]==0 & lista_datos[1]==1 & lista_datos[2]==1 & lista_datos[3]==1):
            self.conteo_m_A_c_m=self.conteo_m_A_c_m+1
            self.alpha_A_c_m[1]=self.alpha_A_c_m[1]+self.conteo_m_A_c_m
            
        if(lista_datos[0]==1 & lista_datos[1]==0 & lista_datos[2]==0 & lista_datos[3]==1):
            self.conteo_m_B_f_v=self.conteo_m_B_f_v+1
            self.alpha_B_f_v[1]=self.alpha_B_f_v[1]+self.conteo_m_B_f_v
            
        if(lista_datos[0]==1 & lista_datos[1]==0 & lista_datos[2]==1 & lista_datos[3]==1):
            self.conteo_m_B_f_m=self.conteo_m_B_f_m+1
            self.alpha_B_f_m[1]=self.alpha_B_f_m[1]+self.conteo_m_B_f_m
            
        if(lista_datos[0]==1 & lista_datos[1]==1 & lista_datos[2]==0 & lista_datos[3]==1):
            self.conteo_m_B_c_v=self.conteo_m_B_c_v+1
            self.alpha_B_c_v[1]=self.alpha_B_c_v[1]+self.conteo_m_B_c_v
            
        if(lista_datos[0]==1 & lista_datos[1]==1 & lista_datos[2]==1 & lista_datos[3]==0):
            self.conteo_m_B_c_m=self.conteo_m_B_c_m+1
            self.alpha_B_c_m[1]=self.alpha_B_c_m[1]+self.conteo_m_B_c_m
        
        
        self.beliefs=self.belief_formation()
        
    def get_reward(self):
        return(self.recompensa)
            
    

In [ ]:
class random_agent:
    def __init__(self,true_model,model_simulator, max_variables, max_valores):
        self.recompensa=[0]
        self.simulator=model_simulator.action_simulator
    
    def make_decision(self):
        p=list(np.random.random_sample(1))
        if (p[0] > 0.5):
            return(0)
        else:
            return(1)
        
    def observing_external(self):
        chosen_action=self.make_decision()
        datos=self.simulator(chosen_action)
        self.recompensa.append(datos['Final'])
        return(datos)
    
    def get_reward(self):
        return(self.recompensa)

In [ ]:
class Q_learning_agent:
    def __init__(self, model_simulator, max_valores,epsilon):
        self.epsilon = epsilon
        self.simulator=model_simulator.action_simulator
        self.K=max_valores
        self.k = np.zeros(self.K, dtype=np.int)  # number of times action was chosen
        self.Q = np.zeros(self.K, dtype=np.float)  # estimated value
        self.reward_history = []

    # Update Q action-value using:
    # Q(a) <- Q(a) + 1/(k+1) * (r(a) - Q(a))
    def update_Q(self, action, reward):
        self.k[action] += 1  # update action counter k -> k+1
        self.Q[action] += (1./self.k[action]) * (reward - self.Q[action])

        # Choose action using an epsilon-greedy agent
    def choose_action(self, force_explore=False):
        rand = np.random.random()  # [0.0,1.0)
        if (rand < self.epsilon) or force_explore:
            action_explore = np.random.randint(self.K)  # explore random bandit
            return action_explore
        else:
            #action_greedy = np.argmax(self.Q)  # exploit best current bandit
            action_greedy = np.random.choice(np.flatnonzero(self.Q == self.Q.max()))
            return action_greedy
    
    def observe_external(self):
        chosen_action=self.choose_action()
        datos=self.simulator(chosen_action)
        reward=datos['Final']
        self.update_Q(chosen_action,reward)
        self.reward_history.append(reward)
        
    def get_reward(self):
        return(self.reward_history)
        

### Uso

In [ ]:
modelo=true_causal_model(4,infer,sampling,model)


In [ ]:
agente_random=random_agent(model,modelo,4,2)
#                (self, true_model,model_simulator, max_variables, max_valores)

In [ ]:
agente=causal_agent(model,modelo,4,2)

In [ ]:
agente.belief_formation()

In [ ]:
agente.print_beliefs()

In [ ]:
agente.fix_causal_model()

In [ ]:
agente.belief_updating()

In [ ]:
agente.fix_causal_model()

In [ ]:
print(agente_causal.local_model.get_cpds('Enfermedad'))

In [ ]:
agente.get_reward()

## Experimentos numéricos

### Causal agent

In [ ]:
N=100
modelo=true_causal_model(4,infer,sampling,model)
agente_causal=causal_agent(model,modelo,4,2)
agente_causal.belief_formation()
recompensa_causal=np.zeros(N,dtype='float')
recompensa_promedio_causal=np.zeros(N,dtype='float')
for i in range(N):
    agente_causal.belief_updating()
    agente_causal.fix_causal_model()
recompensas_causal=agente_causal.get_reward()


In [ ]:
print(agente_causal.local_model.get_cpds('Enfermedad'))

In [ ]:
plt.plot(recompensas_causal)
plt.xlim([1, N])
plt.ylim([0, 5])

In [ ]:
recompensa_acumulada_causal=np.cumsum(recompensas_causal)
recompensa_promedio_causal=recompensa_acumulada_causal/N
plt.plot(recompensa_promedio_causal)
plt.xlabel("Number Rounds")
plt.ylabel("Average Reward for Causal Agent".format())

In [ ]:
recompensa_acumulada_causal

### Random agent

In [ ]:

modelo=true_causal_model(4,infer,sampling,model)
agente_random=random_agent(model,modelo,4,2)
recompensa_random=np.zeros(N,dtype='float')
recompensa_promedio_random=np.zeros(N,dtype='float')
for i in range(N):
    agente_random.observing_external()
recompensas_random=agente_random.get_reward()

In [ ]:
plt.plot(recompensas_random)
plt.xlim([1, N])
plt.ylim([0, 5])

In [ ]:
recompensa_acumulada_random=np.cumsum(recompensas_random)
recompensa_promedio_random=recompensa_acumulada_random/N
plt.plot(recompensa_promedio_random)
plt.xlabel("Rounds for random agent")
plt.ylabel("Average Reward for random agent".format())

### Q-Learning

In [ ]:
modelo=true_causal_model(4,infer,sampling,model)
agente_Q=Q_learning_agent(modelo,2,0.3)
recompensa_Q=np.zeros(N,dtype='float')
recompensa_Q=np.zeros(N,dtype='float')
for i in range(N):
    agente_Q.observe_external()
    
recompensas_Q=agente_Q.get_reward()

In [ ]:
plt.plot(recompensas_Q)
plt.xlim([1, N])
plt.ylim([0, 5])

In [ ]:
recompensa_acumulada_Q=np.cumsum(recompensas_Q)
recompensa_promedio_Q=recompensa_acumulada_Q/N
plt.plot(recompensa_promedio_Q)
plt.xlabel("Rounds for Q-learning agent")
plt.ylabel("Average Reward for Q-learning agent".format())

### Todos juntos

In [ ]:
plt.plot(recompensa_promedio_causal, label='Causal agent')
plt.plot(recompensa_promedio_random,linestyle=':',label='Random agent')
plt.plot(recompensa_promedio_Q,linestyle='--',label='Q agent')
leg = plt.legend(loc='upper left', shadow=True, fontsize=10)
plt.xlim([1, N])
plt.ylim([0, 1])
plt.ylabel("Average Reward for agent")
plt.xlabel("Rounds")

In [ ]:
recompensa_promedio_causal

In [ ]:
recompensa_acumulada_causal

In [ ]:
agente_informado=fully_informed_agent(modelo)

In [ ]:
agente_informado.make_decision()

In [ ]:
agente_informado.observing_external()

In [ ]:
agente_informado.get_reward()

In [ ]:
N=20
modelo=true_causal_model(4,infer,sampling,model)
agente_informado=fully_informed_agent(modelo)
recompensa_informada=np.zeros(N,dtype='float')
recompensa_promedio_informada=np.zeros(N,dtype='float')
for i in range(N):
    agente_informado.observing_external()
recompensas_informado=agente_informado.get_reward()

In [ ]:
recompensas_informado

In [ ]:
plt.plot(recompensas_informado)

In [ ]:
recompensa_acumulada_informado=np.cumsum(recompensas_informado)
recompensa_promedio_informado=recompensa_acumulada_informado/N

In [ ]:
plt.plot(recompensa_promedio_informado)

### Evaluando el modelo causal aprendido


In [ ]:
print(agente_causal.local_model.get_cpds('Enfermedad'))

In [ ]:
print(agente_causal.local_model.get_cpds('Tratamiento'))

In [ ]:
print(agente_causal.local_model.get_cpds('Final'))

In [ ]:
print(model.get_cpds('Final'))

### Prueba de la creacion del modelo

In [ ]:
modelo=true_causal_model(4,infer,sampling,model)
agente_causal=causal_agent(model,modelo,4,2)
agente_causal.belief_formation()
print(agente_causal.local_model.get_cpds('Enfermedad'))

In [ ]:
agente_causal.belief_updating()

In [ ]:
agente_causal.fix_causal_model()

In [ ]:
print(agente_causal.local_model.get_cpds('Enfermedad'))

In [ ]:
elements = [0,1]
probabilities = [0.4,0.6]
res=np.random.choice(elements, 1, p=probabilities)
res[0]

In [ ]:
np.random.randint(0,2)

In [ ]:
agente_causal.print_beliefs()

In [ ]:
modelo.action_simulator(1)